In [2]:
import pandas as pd
import tensorflow
import keras
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
data=pd.read_csv('/Users/pranav/Desktop/Coding/Data-Science/DATA/IMDB Dataset.csv')
encoder=LabelEncoder()

data['sentiment']=encoder.fit_transform(data['sentiment'])

def fun(x):
    x=str(x).lower()
    x=re.sub(r'<[^>]+>','',x)
    x=re.sub(r'[^\w\s]+','',x)
    return x

data['review']=data['review'].apply(fun)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer(num_words=1000,oov_token="<OOV>")
tokenizer.fit_on_texts(data['review'])
sequences=tokenizer.texts_to_sequences(data['review'])

pad_sequence=pad_sequences(sequences,maxlen=200,padding='post',truncating='post')

In [4]:
X_train,X_test,y_train,y_test=train_test_split(pad_sequence,data['sentiment'])

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,SimpleRNN,LSTM,GRU,Embedding,Bidirectional

In [24]:
model=Sequential([
    Embedding(input_dim=1000,output_dim=8,input_length=200),
    LSTM(32,return_sequences=True),
    Dense(1,activation='sigmoid')
])

model.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])
model.fit(
    X_train,y_train,epochs=5,validation_data=(X_test,y_test),batch_size=32,validation_split=0.42
)

Epoch 1/5
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 24s 20ms/step - accuracy: 0.5133 - loss: 0.6918 - val_accuracy: 0.5316 - val_loss: 0.6861
Epoch 2/5
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 24s 21ms/step - accuracy: 0.5419 - loss: 0.6823 - val_accuracy: 0.4990 - val_loss: 0.6955
Epoch 3/5
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 24s 20ms/step - accuracy: 0.5242 - loss: 0.6883 - val_accuracy: 0.5367 - val_loss: 0.6898
Epoch 4/5
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 24s 20ms/step - accuracy: 0.5645 - loss: 0.6746 - val_accuracy: 0.5462 - val_loss: 0.6792
Epoch 5/5
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 24s 20ms/step - accuracy: 0.6289 - loss: 0.6368 - val_accuracy: 0.7226 - val_loss: 0.5503


In [25]:
model.evaluate(X_test,y_test)
y_pred=(model.predict(X_test)>0.5)

from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred,y_test)

391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7215 - loss: 0.5516
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


array([[3193,  425],
       [3042, 5840]])